### Rajas Thakur - rxt190000

### Acknowledgement
Notebook inspired from following sources :
1. https://www.analyticsvidhya.com/blog/2018/07/hands-on-sentiment-analysis-dataset-python/

2. https://medium.com/towards-artificial-intelligence/emoticon-and-emoji-in-text-mining-7392c49f596a

3. https://towardsdatascience.com/sentiment-analysis-with-text-mining-13dd2b33de27

In [6]:
#Checking current directory
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks')

In [8]:
#Importing necessary libraries
import nltk
import spacy
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
#from TokenizationTest import TokenizationTest
#from happyfuntokenizing import Tokenizer as potts
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn import linear_model
import pandas as pd
import regex as re
import numpy as np

In [9]:
train = pd.read_csv("train.csv",encoding="ISO-8859-1")
test = pd.read_csv("test.csv",encoding="ISO-8859-1")

In [10]:
train.head()

,Id,text,Target
0,1,@USAirways ! THE WORST in customer service. @...,-1
1,2,@united call wait times are over 20 minutes an...,-1
2,3,@JetBlue what's up with the random delay on fl...,-1
3,4,@AmericanAir Good morning! Wondering why my p...,0
4,5,@united UA 746. Pacific Rim and Date Night cut...,-1


In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7320 entries, 0 to 7319
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Id      7320 non-null   int64 
 1   text    7320 non-null   object
 2   Target  7320 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 171.7+ KB


In [12]:
test.head()

,id,text
0,7322,@AmericanAir In car gng to DFW. Pulled over 1h...
1,7323,"@AmericanAir after all, the plane didnÂÃÂªt ..."
2,7324,@SouthwestAir can't believe how many paying cu...
3,7325,@USAirways I can legitimately say that I would...
4,7326,@AmericanAir still no response from AA. great ...


In [13]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7320 entries, 0 to 7319
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      7320 non-null   int64 
 1   text    7320 non-null   object
dtypes: int64(1), object(1)
memory usage: 114.5+ KB


**Removing @ mentions from the tweets**


In [14]:
# We will define a function to remove special characters specified to the function
def pattern_removal(input,pattern):
  s = re.findall(pattern,input)
  for i in s:
    input = re.sub(i,'',input)
  return input

In [15]:
train['text'] = np.vectorize(pattern_removal)(train['text'],"@[\w]*")

In [16]:
train.head()

,Id,text,Target
0,1,! THE WORST in customer service. ! Calling ...,-1
1,2,call wait times are over 20 minutes and airpo...,-1
2,3,what's up with the random delay on flight 172...,-1
3,4,Good morning! Wondering why my pre-TSA check...,0
4,5,UA 746. Pacific Rim and Date Night cut out. N...,-1


In [17]:
test['text'] = np.vectorize(pattern_removal)(test['text'],"@[\w]*")

In [18]:
test.head()

,id,text
0,7322,In car gng to DFW. Pulled over 1hr ago - very...
1,7323,"after all, the plane didnÂÃÂªt land in iden..."
2,7324,can't believe how many paying customers you l...
3,7325,I can legitimately say that I would have rath...
4,7326,still no response from AA. great job guys!


**Converting to Lower Case**

In [19]:
train['text']=train['text'].str.lower()

In [20]:
test['text']=test['text'].str.lower()

**Removing http links**

In [21]:
# We will use str.replace function to replace all http and https links with blanks in the tweets
train['text'] = train['text'].str.replace("http[s]\S+", "")

In [22]:
train['text'].iloc[94]

'  wow what an amazing video  great world guys #bluemanity #corevalues #passion #aerojobmarket'

In [23]:
test['text'] = test['text'].str.replace("http[s]\S+", "")

In [24]:
test['text'].iloc[11]

' it is really embarrassing when asking for complimentary drink/snack detailed here:  &amp; being argued with.'

**Converting Emoticons to relevant text**

In [25]:
#Installing emot library
!pip install emot
#Importing libraries
import re
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

You should consider upgrading via the 'c:\python\python37\python.exe -m pip install --upgrade pip' command.


In [26]:
# Function for converting emoticons into word
def convert_emoticons(text):
    for e in EMOTICONS:
        text = re.sub(u'('+e+')', "_".join(EMOTICONS[e].replace(",","").split()), text)
    return text

In [27]:
train['text'] = train['text'].apply(convert_emoticons)

In [28]:
train['text'].iloc[46]

' credit to you for replying.  iâ\x89ã\x9bâªll look forward to hopefully one of those 103 on my flight home tonight Happy_face_or_smiley'

In [29]:
#Replacing the _ in converted emoticons by space
train['text'] = train['text'].str.replace('_',' ')

In [30]:
test['text'] = test['text'].apply(convert_emoticons)

In [31]:
test['text'] = test['text'].str.replace('_',' ')

**Removing Special Characters**

In [32]:
# All the special characters and words starting with & are removed here 
train['text'] = np.vectorize(pattern_removal)(train['text'],"&[\w]*")
train['text'] = train['text'].str.replace("[^a-zA-Z\.]"," ")

In [33]:
train['text'].iloc[96]

' make sure you take care of  team    team. we are         '

In [34]:
test['text'] = np.vectorize(pattern_removal)(test['text'],"&[\w]*")
test['text'] = test['text'].str.replace("[^a-zA-Z\.]"," ")

In [35]:
test.head()

,id,text
0,7322,in car gng to dfw. pulled over hr ago very...
1,7323,after all the plane didn t land in iden...
2,7324,can t believe how many paying customers you l...
3,7325,i can legitimately say that i would have rath...
4,7326,still no response from aa. great job guys


**Replacing Period with Space**

In [36]:
train['text'] = train['text'].str.replace("\."," ")

In [37]:
test['text'] = test['text'].str.replace("\."," ")

**Removing Extra Spaces**

In [38]:
# We will remove extra spaces and replace them with single space
train['text'] = train['text'].apply(lambda x:re.sub('\s+',' ',x))

In [39]:
test['text'] = test['text'].apply(lambda x:re.sub('\s+',' ',x))

**Removing Stop Words**

In [559]:
nltk.download('stopwords')
from nltk.corpus import stopwords as nltk_stopwords
stop = set(nltk_stopwords.words('english'))
whitelist = ["n't", "not", "no"]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [560]:
train['text'] = train['text'].apply(lambda x: ' '.join([word for word in x.split() if (word not in (stop) or word in whitelist)]))

In [561]:
train.head()

,Id,text,Target
0,1,worst customer service calling month book flig...,-1
1,2,call wait times minutes airport wait times longer,-1
2,3,random delay flight chance false alarm,-1
3,4,good morning wondering pre tsa check not board...,0
4,5,ua pacific rim date night cut not constantly r...,-1


In [562]:
test['text'] = test['text'].apply(lambda x: ' '.join([word for word in x.split() if (word not in (stop) or word in whitelist)]))

In [563]:
test.head()

,id,text
0,7322,car gng dfw pulled hr ago icy roads hold aa si...
1,7323,plane land identical worseConfusion conditions...
2,7324,believe many paying customers left high dry no...
3,7325,legitimately say would rather driven cross cou...
4,7326,still no response aa great job guys


**Lemmatization**

In [495]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [496]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [497]:
def lemmatization_function(text):
  tokens = text.split()
  lemm_tokens = [lemmatizer.lemmatize(x) for x in tokens]
  return ' '.join(lemm_tokens)

In [498]:
def lemmatization_function_spacy(text):
  tokens = nlp(text)
  lemm_tokens = [x.lemma_ for x in tokens]
  return ' '.join(lemm_tokens)

In [499]:
#train['text'] = train['text'].apply(lemmatization_function)
train['text'] = train['text'].apply(lemmatization_function_spacy)

In [500]:
#test['text'] = test['text'].apply(lemmatization_function)
test['text'] = test['text'].apply(lemmatization_function_spacy)

In [443]:
train.head()

,Id,text,Target
0,1,bad customer service call month book flight po...,-1
1,2,call wait times minutes airport wait time longer,-1
2,3,random delay flight chance false alarm,-1
3,4,good morning wonder pre tsa check board pass m...,0
4,5,pacific date night constantly randomly spot re...,-1


In [ ]:
train['text'].iloc[3]

'  good morning    wonder why -PRON- pre tsa check be not on -PRON- boarding pass this morning  '

**Stemming**

In [40]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer 
stemmer = PorterStemmer()

In [41]:
def stemming_function(text):
  tokens = text.split()
  stem_tokens = [stemmer.stem(x) for x in tokens]
  return ' '.join(stem_tokens)

In [42]:
train['text'] = train['text'].apply(stemming_function)

In [43]:
test['text'] = test['text'].apply(stemming_function)

In [463]:
train['text'].iloc[3]

'good morning wonder pre tsa check board pass morning'

In [464]:
train.head()

,Id,text,Target
0,1,bad customer service call month book flight po...,-1
1,2,call wait times minutes airport wait time longer,-1
2,3,random delay flight chance false alarm,-1
3,4,good morning wonder pre tsa check board pass m...,0
4,5,pacific date night constantly randomly spot re...,-1


## Creating pipelines for ML models

In [45]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier

In [46]:
# Splitting the dataset into train and test sets
from sklearn.model_selection import train_test_split
X = train['text']
y = train['Target']
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.10, random_state=0)

#### Logistic Regression - TFIDF Vectorizer

In [901]:
pipe = Pipeline([('tfidf',TfidfVectorizer()),
                     ('LR',LogisticRegression(max_iter=10000,solver='lbfgs',multi_class='multinomial'))])

param_grid = {'LR__C': [0.01, 0.1,0.5, 1],
              'LR__penalty':['l1','l2'],
              'tfidf__max_df':[0.1,0.5,0.75,0.95],
              'tfidf__min_df':[1,2,5],
              'tfidf__ngram_range':[(1,1),(1,2),(2,2)]}

In [902]:
LR_TF = GridSearchCV(pipe, param_grid, cv=5,n_jobs=-1,scoring="f1_macro")

In [903]:
LR_TF.fit(X_train, y_train)
print("Best cross-validation score: {:.2f}".format(LR_TF.best_score_))

Best cross-validation score: 0.70


In [904]:
LR_TF.best_params_

{'LR__C': 1,
 'LR__penalty': 'l2',
 'tfidf__max_df': 0.5,
 'tfidf__min_df': 5,
 'tfidf__ngram_range': (1, 1)}

In [905]:
print("Train score: {:.2f}".format(LR_TF.score(X_train, y_train)))
print("Test score: {:.4f}".format(LR_TF.score(X_test, y_test)))

Train score: 0.81
Test score: 0.6960


In [906]:
y_pred = LR_TF.predict(X_test)

In [907]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.82      0.95      0.88       475
           0       0.63      0.48      0.54       134
           1       0.84      0.55      0.67       123

    accuracy                           0.80       732
   macro avg       0.76      0.66      0.70       732
weighted avg       0.79      0.80      0.78       732



#### Logistic Regression - Count Vectorizer

In [47]:
pipe = Pipeline([('count',CountVectorizer()),
                     ('LR',LogisticRegression(max_iter=10000,solver='lbfgs',multi_class='multinomial'))])

param_grid = {'LR__C': [0.001, 0.01, 0.1, 1, 10],
              'LR__penalty':['l1','l2'],
              'count__max_df':[0.2,0.5,0.75,0.95],
              'count__min_df':[0.1,0.5,1],
              'count__ngram_range':[(1,1),(1,2),(2,2)]}

In [48]:
LR_CV = GridSearchCV(pipe, param_grid, cv=5,n_jobs=-1,scoring="f1_macro")

In [49]:
LR_CV.fit(X_train, y_train)
print("Best cross-validation score: {:.4f}".format(LR_CV.best_score_))

Best cross-validation score: 0.7360


In [50]:
LR_CV.best_params_

{'LR__C': 1,
 'LR__penalty': 'l2',
 'count__max_df': 0.5,
 'count__min_df': 1,
 'count__ngram_range': (1, 2)}

In [51]:
print("Train score: {:.2f}".format(LR_CV.score(X_train, y_train)))
print("Test score: {:.4f}".format(LR_CV.score(X_test, y_test)))

Train score: 0.99
Test score: 0.7306


In [52]:
y_pred = LR_CV.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.86      0.93      0.89       475
           0       0.62      0.55      0.58       134
           1       0.81      0.64      0.72       123

    accuracy                           0.81       732
   macro avg       0.76      0.71      0.73       732
weighted avg       0.81      0.81      0.81       732



In [900]:
y_predt = LR_CV.predict(test['text'])
test['target']=y_predt
test.to_csv('FinalSubmission_withstopwords.csv',index=False)

#### Logistic Regression - Hash Vectorizer

In [212]:
from sklearn.feature_extraction.text import HashingVectorizer

pipe = Pipeline([('hash',HashingVectorizer(n_features=10000)),
                     ('LR',LogisticRegression(max_iter=10000,solver='lbfgs',multi_class='multinomial'))])

param_grid = {'LR__C': [0.001, 0.01, 0.1, 1, 10],
              'hash__ngram_range':[(1,1),(1,2),(2,2)]}

In [213]:
LR_HV = GridSearchCV(pipe, param_grid, cv=5,n_jobs=-1,scoring="f1_macro")

In [214]:
LR_HV.fit(X_train, y_train)
print("Best cross-validation score: {:.2f}".format(LR_HV.best_score_))

Best cross-validation score: 0.68


In [215]:
LR_HV.best_params_

{'LR__C': 10, 'hash__ngram_range': (1, 1)}

In [216]:
print("Train score: {:.2f}".format(LR_HV.score(X_train, y_train)))
print("Test score: {:.4f}".format(LR_HV.score(X_test, y_test)))

Train score: 0.91
Test score: 0.6707


In [217]:
y_pred = LR_HV.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.83      0.89      0.86       475
           0       0.53      0.49      0.51       134
           1       0.71      0.59      0.64       123

    accuracy                           0.77       732
   macro avg       0.69      0.66      0.67       732
weighted avg       0.76      0.77      0.76       732



#### Naive Bayes - TFIDF Vectorizer

In [218]:
pipe = Pipeline([('tfidf',TfidfVectorizer()),
                     ('NB',MultinomialNB(fit_prior=True, class_prior=None))])

param_grid = {'NB__alpha': [0.1,0.25,0.5,0.75],
              'tfidf__max_df':[0.2,0.5,0.7,0.9],
              'tfidf__min_df':[0.1,1,2,5],
              'tfidf__ngram_range':[(1,1),(1,2),(2,2)]}

In [219]:
NB_TF = GridSearchCV(pipe, param_grid, cv=5,n_jobs=-1,scoring="f1_macro")

In [220]:
NB_TF.fit(X_train, y_train)
print("Best cross-validation score: {:.2f}".format(NB_TF.best_score_))

Best cross-validation score: 0.62


In [221]:
print("Train score: {:.2f}".format(NB_TF.score(X_train, y_train)))
print("Test score: {:.2f}".format(NB_TF.score(X_test, y_test)))

Train score: 0.86
Test score: 0.63


In [222]:
y_pred = NB_TF.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.78      0.94      0.85       475
           0       0.55      0.32      0.41       134
           1       0.80      0.52      0.63       123

    accuracy                           0.76       732
   macro avg       0.71      0.59      0.63       732
weighted avg       0.74      0.76      0.73       732



#### Naive Bayes - Count Vectorizer

In [223]:
pipe = Pipeline([('count',CountVectorizer()),
                     ('NB',MultinomialNB(fit_prior=True, class_prior=None))])

param_grid = {'NB__alpha': [0,0.001,0.1,0.5,0.7,1],
              'count__max_df':[0.2,0.5,0.7,0.9],
              'count__min_df':[0.1,0.5,1,2],
              'count__ngram_range':[(1,1),(1,2),(2,2)]}

In [224]:
NB_CV = GridSearchCV(pipe, param_grid, cv=5,n_jobs=-1,scoring="f1_macro")

In [225]:
NB_CV.fit(X_train, y_train)
print("Best cross-validation score: {:.2f}".format(NB_CV.best_score_))

Best cross-validation score: 0.67


In [226]:
print("Train score: {:.2f}".format(NB_CV.score(X_train, y_train)))
print("Test score: {:.2f}".format(NB_CV.score(X_test, y_test)))

Train score: 0.79
Test score: 0.68


In [227]:
y_pred = NB_CV.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.83      0.88      0.85       475
           0       0.54      0.44      0.49       134
           1       0.72      0.67      0.70       123

    accuracy                           0.77       732
   macro avg       0.70      0.67      0.68       732
weighted avg       0.76      0.77      0.76       732



In [500]:
y_predt = NB_CV.predict(test['text'])
test['target']=y_predt
test.to_csv('predictionNB_CV5.csv',index=False)

#### Naive Bayes - Hash Vectorizer

In [228]:
pipe = Pipeline([('hash',HashingVectorizer(n_features=10000,alternate_sign=False)),
                     ('NB',MultinomialNB(fit_prior=True, class_prior=None))])

param_grid = {'NB__alpha': [0.001,0.1,0.5,0.7,1],
              'hash__ngram_range':[(1,1),(1,2),(2,2)]}

In [229]:
NB_HV = GridSearchCV(pipe, param_grid, cv=5,n_jobs=-1,scoring="f1_macro")

In [230]:
NB_HV.fit(X_train, y_train)
print("Best cross-validation score: {:.2f}".format(NB_HV.best_score_))

Best cross-validation score: 0.59


In [231]:
NB_HV.best_params_

{'NB__alpha': 0.001, 'hash__ngram_range': (1, 1)}

In [232]:
print("Train score: {:.2f}".format(NB_HV.score(X_train, y_train)))
print("Test score: {:.4f}".format(NB_HV.score(X_test, y_test)))

Train score: 0.83
Test score: 0.5811


In [233]:
y_pred = NB_HV.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.76      0.92      0.83       475
           0       0.52      0.34      0.41       134
           1       0.68      0.39      0.49       123

    accuracy                           0.73       732
   macro avg       0.65      0.55      0.58       732
weighted avg       0.70      0.73      0.70       732



#### XGBoost - TFIDF Vectorizer

In [663]:
from xgboost import XGBClassifier
pipe = Pipeline([('tfidf',TfidfVectorizer()),
                     ('XGB',XGBClassifier(objective='multi:softmax',seed=0))])

param_grid = {'XGB__learning_rate':[0.01,0.1,0.5,1],
              'XGB__max_depth':[2,3,5,7],
              'tfidf__max_df':[0.2,0.5,0.9],
              'tfidf__min_df':[1,2],
              'tfidf__ngram_range':[(1,1),(1,2),(2,2)]}

In [666]:
XGB_TF = GridSearchCV(pipe, param_grid, cv=5,n_jobs=-1,scoring="f1_macro")

In [667]:
XGB_TF.fit(X_train, y_train)
print("Best cross-validation score: {:.2f}".format(XGB_TF.best_score_))

Best cross-validation score: 0.67


In [668]:
XGB_TF.best_params_

{'XGB__learning_rate': 0.5,
 'XGB__max_depth': 7,
 'tfidf__max_df': 0.5,
 'tfidf__min_df': 1,
 'tfidf__ngram_range': (1, 1)}

In [669]:
print("Train score: {:.2f}".format(XGB_TF.score(X_train, y_train)))
print("Test score: {:.2f}".format(XGB_TF.score(X_test, y_test)))

Train score: 0.92
Test score: 0.66


In [670]:
y_pred = XGB_TF.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.79      0.87      0.83      1370
           0       0.54      0.45      0.49       461
           1       0.71      0.59      0.65       365

    accuracy                           0.74      2196
   macro avg       0.68      0.64      0.66      2196
weighted avg       0.73      0.74      0.73      2196



#### XGBoost - Count Vectorizer

In [48]:
from xgboost import XGBClassifier
pipe = Pipeline([('count',CountVectorizer()),
                     ('XGB',XGBClassifier())])

param_grid = {'XGB__learning_rate':[0.01,0.1,0.5,1],
              'XGB__max_depth':[2,3,5,7],
              'count__max_df':[0.2,0.5,0.7,0.9],
              'count__min_df':[1,2],
              'count__ngram_range':[(1,1),(1,2),(2,2)]}

In [49]:
XGB_CV = GridSearchCV(pipe, param_grid, cv=5,n_jobs=-1,scoring="f1_macro")

In [50]:
XGB_CV.fit(X_train, y_train)
print("Best cross-validation score: {:.2f}".format(XGB_CV.best_score_))

Best cross-validation score: 0.68


In [51]:
XGB_CV.best_params_

{'XGB__learning_rate': 1,
 'XGB__max_depth': 5,
 'count__max_df': 0.2,
 'count__min_df': 1,
 'count__ngram_range': (1, 2)}

In [52]:
print("Train score: {:.2f}".format(XGB_CV.score(X_train, y_train)))
print("Test score: {:.2f}".format(XGB_CV.score(X_test, y_test)))

Train score: 0.86
Test score: 0.67


In [55]:
y_pred = XGB_CV.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.80      0.86      0.83      1370
           0       0.56      0.46      0.51       461
           1       0.71      0.63      0.67       365

    accuracy                           0.74      2196
   macro avg       0.69      0.65      0.67      2196
weighted avg       0.73      0.74      0.73      2196



### Choosing the final model

#### The best model suitable for this dataset will be with
1. Keeping Stopwords
2. Using Porter stemmer
3. Using Count Vectorizer with best max_df = 0.5 ,min_df = 1 and ngram_range =(1,2) 
4. Using Logistic regression model with best parameters as C=1 and penalty =l2

#### Although the final model overfits the dataset, the F1 score for each class is the highest with this model and the cross validation score for the overall  model is the highest ,hence I have chosen this model.

In [56]:
# Making final predictions
y_predt = LR_CV.predict(test['text'])
test['target']=y_predt
test.drop(columns='text',inplace = True)
test.to_csv('FinalSubmission.csv',index=False)